# ETL

En esta jupyter notebook se desarrolla la extracción, transformación y carga de los conjuntos de datos.

# Importaciones

In [1]:
import pandas as pd
import json
import gzip
import numpy as np
import ast
from tqdm import tqdm


%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

# Extracción de los datos

Funcion para extraer los datos ".json" y convertirlos en ".csv" para la exploración.

In [6]:

def procesar_archivo_json(ruta, tipo='json', nombre_archivo_salida='datos.csv'):
    """
    Esta función lee datos desde cada archivo de mi carpeta y los convierte en un DataFrame de pandas.

    Args:
        ruta (str): La ruta al archivo que se va a leer.
        tipo (str, optional): El formato de los datos en el archivo. Puede ser 'json' o 'literal'. 
                              Si es 'json', los datos se cargarán utilizando json.loads. 
                              Si es 'literal', los datos se cargarán utilizando ast.literal_eval. 
                              Por defecto es 'json'.
        nombre_archivo_salida (str, optional): El nombre del archivo CSV de salida.
                                                Por defecto es 'datos.csv'.

    Returns:
        None
    """
    filas = []  # Será la lista para almacenar cada fila de datos

    with gzip.open(ruta, 'rt', encoding='utf-8') as f:
        for line in tqdm(f.readlines(), desc=f"Leyendo {ruta}"):
            if tipo == 'json':
                data = json.loads(line)
            elif tipo == 'literal':
                data = ast.literal_eval(line)
            filas.append(data)

    # Convierte la lista de filas en un DataFrame de pandas
    df = pd.DataFrame(filas)
    
    # Guarda el DataFrame como un archivo CSV
    df.to_csv(nombre_archivo_salida, index=False)

In [42]:
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz', nombre_archivo_salida='games.csv' )
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\users_items.json.gz', tipo= 'literal',nombre_archivo_salida='items.csv' )
#No se va a procesar el archivo "user_reviews.json" ya que presentó inconsistencias las cuales tienen que ser tratadas en formato json

Leyendo C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz: 100%|██████████| 120445/120445 [00:00<00:00, 143771.85it/s]


## Extrayendo archivos de "user_reviews"

In [26]:
file_path = 'australian_user_reviews.json'

data_list1 =[]
#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list1.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list1)
data_it1 = data_it.explode(['reviews'])
data_it2 = pd.json_normalize(data_it1['reviews']).set_index(data_it1['reviews'].index)
data_it3= pd.concat([data_it2, data_it1],axis=1)
data_it3.to_csv('data/reviews.csv')

# Transformación de los datos


## 'games.csv'.


In [43]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\games.csv', encoding = "utf-8")
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 1169016


In [45]:
df = df.dropna()
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 0


##### Que columnas conservar


Teniendo en cuenta que se nos pide : 

* def *developer*( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
Mantener : "tags", "developer"

* def *userdata*( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Mantener : "price"

* def *UserForGenre*( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Mantener : "genre", "release_date"

* def *best_developer_year*( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Mantener : "developer"

* def *developer_reviews_analysis*( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Mantener : "developer".

Las columnas a mantener: "developer", "price", "genre", "release_date", "tags", "id", "title". A eliminar: "app_name", "publisher", "url", "early_access", "specs"

In [46]:
del df['app_name']
del df['publisher']
del df['url']
del df['early_access']
del df['specs']

In [47]:
df.columns

Index(['genres', 'title', 'release_date', 'tags', 'reviews_url', 'price', 'id',
       'developer'],
      dtype='object')

In [48]:
df.head()

,genres,title,release_date,tags,reviews_url,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,Free To Play,643980.0,Secret Level SRL
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,Free to Play,670290.0,Poolians.com
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,0.99,767400.0,彼岸领域
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,3.99,772540.0,Trickjump Games Ltd


In [49]:
#Convertimos la columa release_date a formato fecha
df['release_date']=pd.to_datetime(df['release_date'], errors='coerce', exact=False)

#Solo nos interesa el año

# Extraer el año y crear una nueva columna 'release_year'
df['year'] = df['release_date'].dt.year.astype('Int64')

# Eliminar la columna 'release_date'
df = df.drop(columns=['release_date'])

In [50]:
# Si aún hay valores nulo, se llenan con la mediana. 
print('cantidad de nulos en "year" :', df['year'].isna().sum())
df['year'] = df['year'].fillna(df['year'].median())
print('cantidad de nulos en "year" :', df['year'].isna().sum())


cantidad de nulos en "year" : 34
cantidad de nulos en "1year" : 0


In [52]:
# Convertir 'price' a tipo numérico, asignar NaN a 'Free To Play'
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [53]:
df.to_csv('steam_games_cleaned.csv', index=False)
df.to_json('steam_games_cleaned.json', orient='records', lines=True)
df.to_parquet('steam_games_cleaned.parquet', index=False)

## 'items.csv

In [53]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\items.csv')
df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [54]:
print("\nEstructura de la columna 'items':")
print(df['items'].iloc[0])


Estructura de la columna 'items':
[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 18

In [55]:
print(df['items'].dtype)

object


### Datos de la Columna "items"

- **item_id**: El identificador único del ítem.
- **item_name**: El nombre del ítem.
- **playtime_forever**: El tiempo total de juego (en minutos) del usuario para este ítem.
- **playtime_2weeks**: El tiempo de juego (en minutos) del usuario en las últimas dos semanas para este ítem.

"items" parece ser una lista de diccionarios en formato json, se procede a desarmar y agregar cada diccionario como columna al df principal

In [56]:
# Suponiendo que 'items' es la columna que contiene las listas de diccionarios como cadenas
for index, row in df.iterrows():
    # Convertir la cadena que representa el diccionario en un diccionario real
    item_list = ast.literal_eval(row['items'])
    # Iterar sobre cada diccionario en la lista
    for item_dict in item_list:
        # Iterar sobre cada par clave-valor en el diccionario
        for key, value in item_dict.items():
            # Agregar las claves del diccionario como columnas
            df.at[index, key] = value

# Eliminar la columna 'items' original si ya no se necesita
df.drop(columns=['items'], inplace=True)

In [57]:
df.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,273350,Evolve Stage 2,58.0,0.0
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,397060,Faeria,0.0,0.0
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,466170,Idling to Rule the Gods,28545.0,1554.0
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,379720,DOOM,1.0,1.0
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,324810,TOXIKK,0.0,0.0


In [58]:
del df['playtime_2weeks']

In [59]:
df.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,273350,Evolve Stage 2,58.0
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,397060,Faeria,0.0
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,466170,Idling to Rule the Gods,28545.0
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,379720,DOOM,1.0
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,324810,TOXIKK,0.0


In [60]:
df.to_csv('data/user_items_limpio.csv', index=False)
df.to_json('data/user_items_limpio.json', orient='records', lines=True)
df.to_parquet('data/user_items_limpio.parquet', index=False)

## 'reviews.csv'

In [27]:
# Importamos pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# El módulo os permite interactuar con el sistema operativo
import os

# Gdown facilita la descarga de archivos desde Google Drive utilizando su ID
import gdown

# JSON es un formato común para el intercambio de datos, y Python tiene soporte incorporado para trabajar con JSON
import json

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
import nltk

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer



In [2]:
#⚠️Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\123la\AppData\Roaming\nltk_data...


True

In [61]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\reviews.csv')
print("Primeras filas del DataFrame:")
df.head()

Primeras filas del DataFrame:


,Unnamed: 0,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,0,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


In [62]:
del df['reviews']

In [63]:
# Calcula valores en la columna 'recommend' 
conteo_por_recomendaciones = df['recommend'].value_counts()
porcentaje_recomendaciones = df['recommend'].value_counts(normalize=True) * 100
#porcentaje_recomendaciones = porcentaje_recomendaciones.round(2).astype(str) + '%'

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_recomendaciones = pd.DataFrame({
    'Conteo': conteo_por_recomendaciones,
    'Porcentaje': porcentaje_recomendaciones.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_recomendaciones = resumen_recomendaciones.sort_values(by='Conteo', ascending=False)
print(resumen_recomendaciones)

           Conteo Porcentaje
recommend                   
True        52473     88.48%
False        6832     11.52%


In [64]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df.replace(['', 'null', 'None'], np.nan, inplace=True)
# Filas donde todas las columnas especificadas tienen valores nulos.
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df[df[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,Unnamed: 0,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
137,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gdxsd,http://steamcommunity.com/id/gdxsd
177,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198094224872,http://steamcommunity.com/profiles/76561198094...
2559,1047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198021575394,http://steamcommunity.com/profiles/76561198021...
10080,3954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cmuir37,http://steamcommunity.com/id/cmuir37
13767,5394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jaysteeny,http://steamcommunity.com/id/Jaysteeny
15493,6135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ML8989,http://steamcommunity.com/id/ML8989
19184,7583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079215291,http://steamcommunity.com/profiles/76561198079...
20223,7952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079342142,http://steamcommunity.com/profiles/76561198079...
25056,9894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198061996985,http://steamcommunity.com/profiles/76561198061...
26257,10381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198108286351,http://steamcommunity.com/profiles/76561198108...


In [65]:
# Se establece umbral del 80% para decidir que columnas eliminar por valores nulos
umbral_nulos = 0.8

# Calcula el porcentaje de valores nulos por columna
porcentaje_nulos = df.isnull().mean()

# Filtra las columnas que superan el umbral
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos]

# Muestra las columnas y su respectivo porcentaje de valores nulos
print("Columnas con más del {}% de valores nulos (candidatas a eliminar):".format(umbral_nulos * 100))
for columna, porcentaje in columnas_a_eliminar.items():
    print("{}: {:.2%}".format(columna, porcentaje))

Columnas con más del 80.0% de valores nulos (candidatas a eliminar):
funny: 86.26%
last_edited: 89.65%


In [66]:

# Eliminamos las columnas que pasan el umbral establecido para valores nulos 
df.drop(columns=columnas_a_eliminar.index, inplace=True)

In [67]:
# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes
df.drop(['helpful','user_url'], axis=1, inplace=True)
# Se buscan registros duplicados
df.sort_values('user_id')
filas_duplicadas = df[df.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
filas_duplicadas.count()

Unnamed: 0    1736
posted        1736
item_id       1736
recommend     1736
review        1736
user_id       1736
dtype: int64

In [68]:
# Se eliminan los registros duplicados basándome en múltiples columnas 
duplicados_eliminados = df.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = filas_duplicadas.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1736


In [69]:
# Asegúrate de que 'posted' sea de tipo datetime
df['posted'] = pd.to_datetime(df['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Crea la columna 'year' a partir de 'posted'
df['year'] = df['posted'].dt.year.astype('Int64')

# Ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df = df.sort_values(['item_id', 'year'])

In [70]:
# Rellenar valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
df['year'] = df.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df['year'] = df['year'].fillna(df['year'].median())
# Se eliminan las columnas 'posted' y 'user_id' que ya no son de utilidad
df.drop(['posted', 'user_id'], axis=1, inplace=True)
df.head()

,Unnamed: 0,item_id,recommend,review,year
5334,2116,10.0,True,this game is the 1# online action game is awes...,2011
22710,8944,10.0,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35556,14404,10.0,True,:D,2012
43156,17642,10.0,True,Good Game :D,2012
24145,9521,10.0,True,jueguenlooooooo,2013


##### Feature Engineering

La columna 'review' también será parte de nuestro estudio ya que incluye reseñas de juegos hechos por distintos usuarios. Partiendo de 'review' se va crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: Debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. De no ser posible este análisis por estar ausente la reseña escrita, tomará el valor de 1. Esta nueva columna reemplaza la de 'review' para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [71]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df.loc[:, 'review'] = df['review'].str.lower()
#Eliminación de caracteres especiales
df['review'] = df['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df.loc[:, 'review'] = df['review'].str.replace('[^\w\s]', '', regex=True)

In [72]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'language'
df['language'] = df['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df['language'].value_counts()
porcentaje_por_idioma = df['language'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,44057,78.53%
pt,2098,3.74%
es,1347,2.4%
so,1089,1.94%
af,973,1.73%


In [73]:
#esto es para tener la información en el EDA

# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
df['idioma_completo'] = df['language'].map(mapeo_idiomas)

# Crear un DataFrame con el resumen de idiomas
resumen_idiomas = df['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)

# Calcular el porcentaje
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(df)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)

# Guardar en un archivo CSV
resumen_idiomas.to_csv('resumen_idiomas.csv', index=False)

In [74]:
# Filtrar los registros donde language no es igual a 'en' 
df = df[df['idioma_completo'] == 'English']

# Borrar la columna 'idioma_completo'
df = df.drop('idioma_completo', axis=1)

In [75]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df.loc[:, 'review'] = df['review'].str.lower()
#Eliminación de caracteres especiales
df['review'] = df['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df.loc[:, 'review'] = df['review'].str.replace('[^\w\s]', '', regex=True)
# Análisis de sentimiento para categorizar las reseñas columna 'review' 

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['review','compound_score'], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage
# Llamar a la función analyze_sentiments
df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(df)

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31179     70.77%
0                     8279     18.79%
1                     4599     10.44%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [76]:
# Borrar la columna 'language'
df = df.drop('language', axis=1)
df

,Unnamed: 0,item_id,recommend,year,sentiment_analysis
5334,2116,10.0,True,2011,2
45529,18755,10.0,True,2013,1
7804,3058,10.0,True,2014,2
7970,3117,10.0,True,2014,2
8522,3332,10.0,True,2014,2
...,...,...,...,...,...
24934,9835,520550.0,True,2014,2
45966,18989,521340.0,True,2014,2
19161,7577,521430.0,True,2014,2
15434,6105,521990.0,True,2014,2


In [77]:
df.to_csv('data/user_reiews_cleaned.csv', index=False)
df.to_json('data/user_reiews_cleaned.json', orient='records', lines=True)
df.to_parquet('data/user_reiews_cleaned.parquet', index=False)